# PART 1 아이템기반 최근접이웃 협업 필터링 실습


1-1. 사용자-영화 평점 행렬 데이터셋
- Grouplens 사이트의 MovieLens 데이터 세트
- https://grouplens.org/datasets/movielens/latest/ 에서 ml-latest-small.zip 다운로드


In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 아이템 기반의 최근접이웃 협업 필터링

- 행 레벨 형태의 원본데이터셋에서 모든 사용자를 row로, 모든 영화를 칼럼으로 구성한 데이터 셋으로 변경
- Pivot_table() 함수를 이용
- Pivot_table(데이터, 인덱스로 지정할 칼럼명, 칼럼으로 지정할 칼럼명)

In [5]:
ratings = ratings[['userId','movieId','rating']]
ratings_matrix = ratings.pivot_table('rating',index = 'userId', columns = 'movieId')
ratings_matrix.head()

# NaN 값은 사용자가 평점을 매기지 않은 영화를 의미 

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 데이터 가공 및 변환
- 칼럼을 title로 변경
- NaN 값을 0으로 변환

In [6]:
# title 컬럼을 얻기 위해 movies와 join
ratings_movies = pd.merge(ratings,movies, on='movieId')
ratings_movies.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [8]:
# columns = 'title'로 변경
ratings_matrix = ratings_movies.pivot_table('rating', index = 'userId', columns = 'title')

# NaN 값을 모두 0으로 변환. 최소 평점이 0.5이기 때문에.
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 영화 간 유사도 산출

- 영화 간의 유사도는 코사인 유사도를 기반으로 측정 
- 사용자가 아닌 영화 간 유사도를 측정해야 하기 때문에 행과 열의 위치를 변경 
- Cosine_similarity() 함수를 통해 코사인 유사도를 구하고, 영화명과 유사도를 매핑하여 DataFrame으로 변환 

In [10]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity()로 반환된 넘파이 행렬을 영화명을 매핑해서 df로 변환
item_sim_df = pd.DataFrame(data=item_sim, 
                           index = ratings_matrix.columns,
                          columns = ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


##### 특정 영화와 유사한 영화 추출

In [13]:
item_sim_df['Inception (2010)'].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

In [14]:
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[1:6]

title
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

### 아이템 기반의 최근접이웃 협업 필터링으로 개인화된 영화 추천

##### 사용자별 최적화된 평점 스코어 예측 - Predict_rating 함수 생성 (전체 영화 대상)

 - Rui는 사용자 u의 모든 영화에 대한 실제 평점과 영화 i의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱한 값 
 - 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 발생 

In [15]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [16]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, 
                                  index = ratings_matrix.index,
                                  columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315


실제 평점과 예측 평점의 차이 (MSE를 통해 측정)
- 예측점수는 평점을 주지 않은 영화에 대해서도 예측했기 때문에, 기존에 평점이 부여된 데이터에 대해서만 측정

In [20]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 MSE를 구함
def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ',get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


##### 사용자별 최적화된 평점 스코어 예측 (Predict_rating_topsim 함수 생성)
- Predict_ratings()함수는 사용자별 예측 평점을 계산하기 위해 다른 모든 영화간의 유사도 벡터를 적용
- 많은 영화의 유사도 벡터를 고려하여 상대적으로 평점 예측이 떨어짐

==>  특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용 
    - Predict_rating()함수에 파라미터 N을 추가하여 가장 유사한 영화의 개수를 설정

In [29]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측행렬 초기화
    pred=np.zeros(ratings_arr.shape)
    
    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행
    for col in range(ratings_arr.shape[1]):
        # 유사도가 큰 순으로 n개의 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:,col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row, col]/= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred
    

In [31]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))
# MSE 가 9,89 에서 3.69로 많이 향상되었음.


ratings_pred_matrix = pd.DataFrame(data=ratings_pred,
                                  index = ratings_matrix.index,
                                  columns = ratings_matrix.columns)

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':


아이템 기반 인접 TOP-20 이웃 MSE:  3.6949717937990285


#### 영화추천을 위해 이미 본 영화를 제외 - get_unseen_movies() 생성

- user_rating이 0보다 크면, 기존에 관람한 영화이기 때문에 이를 already_seen에 저장

In [41]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보를 추출하여 Series로 반환
    # 반환된 user_rating은 title을 index로 가지는 Series객체이다.
    user_rating=ratings_matrix.loc[userId,:]
    
    # 이미 본 영화를 추출하여 list로 
    already_seen = user_rating[user_rating>0].index.tolist()
    # 모든 영화명을 list로
    movies_list = ratings_matrix.columns.tolist()
    # movies_list에서 already_seen 제외
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

최종적으로 사용자에게 영화추천 - recomm_movie_by_userid() 함수 생성

In [44]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [45]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10 )

recomm_movies = pd.DataFrame(data=recomm_movies.values,
                            index = recomm_movies.index,
                            columns = ['pred_score'])
recomm_movies


,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


# PART 2 행렬분해를이용한 잠재요인 협업필터링 실습


In [47]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R행렬에서 널이 아닌 값의 위치 인덱스를 추출하여 실제 R행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [54]:
# 행렬 분해 로직 부분 함수로 만들기

def matrix_factorization(R, K, steps = 200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P=np.random.normal(scale =1./K, size = (num_users, K))
    Q=np.random.normal(scale =1./K, size = (num_items, K))
    
    break_count = 0 
    
    # R>0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]
    
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측값의 차이인 오류 값 구함.
            eij = r-np.dot(P[i,:], Q[j,:].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step%10 ==0):
            print("#### iteration step: ", step, " rmse: ", rmse)
        
    return P,Q

In [52]:
# 사용자-아이템 평점 행렬로 만들기

import numpy as np
import pandas as pd

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index = 'userId', columns='movieId')

# title 컬럼을 얻기 위해 movies와 join
rating_movies = pd.merge(ratings, movies, on='movieId')
# columns='title'로 
ratings_matrix = rating_movies.pivot_table('rating', index = 'userId', columns='title')

#### 행렬 분해를 이용한 잠재요인 협업 필터링 실습
- 사용자-아이템 평점 행렬을 matrix_factorization() 함수를 이용해 행렬분해

In [55]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate = 0.01, r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)
pred_matrix

#### iteration step:  0  rmse:  2.9023619751336867
#### iteration step:  10  rmse:  0.7335768591017927
#### iteration step:  20  rmse:  0.5115539026853442
#### iteration step:  30  rmse:  0.37261628282537446
#### iteration step:  40  rmse:  0.2960818299181014
#### iteration step:  50  rmse:  0.2520353192341642
#### iteration step:  60  rmse:  0.22487503275269854
#### iteration step:  70  rmse:  0.2068545530233154
#### iteration step:  80  rmse:  0.19413418783028685
#### iteration step:  90  rmse:  0.18470082002720403
#### iteration step:  100  rmse:  0.177429275272091
#### iteration step:  110  rmse:  0.1716522696470749
#### iteration step:  120  rmse:  0.16695181946871726
#### iteration step:  130  rmse:  0.16305292191997542
#### iteration step:  140  rmse:  0.15976691929679643
#### iteration step:  150  rmse:  0.1569598699945732
#### iteration step:  160  rmse:  0.15453398186715425
#### iteration step:  170  rmse:  0.15241618551077643
#### iteration step:  180  rmse:  0.1505508073962

array([[3.05508357, 4.09201808, 3.56412973, ..., 2.16108714, 4.01049528,
        0.8594739 ],
       [3.17011871, 3.65799194, 3.30870744, ..., 1.63457611, 4.13573529,
        0.72568398],
       [2.30707338, 1.65885328, 1.44353762, ..., 1.04261842, 2.29389015,
        0.39694143],
       ...,
       [2.15450327, 3.01906005, 2.67937898, ..., 1.9067077 , 2.41956042,
        0.7017386 ],
       [2.56647864, 3.28565903, 2.91012206, ..., 1.60367516, 2.9703819 ,
        0.63687972],
       [3.95178865, 3.54917511, 3.15606904, ..., 1.52882943, 4.11926029,
        0.75123036]])

In [56]:
# 더 쉽게 영화 아이템을 이해하기 위해 영화 타이틀을 컬럼명으로

ratings_pred_matrix = pd.DataFrame(data=pred_matrix,
                                  index = ratings_matrix.index,
                                  columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


##### 개인화된 영화추천(1)
- 9.6 에서 만든  get_unseen_moview() 함수와 recomm_movie_by_userid() 함수를 다시 이용

In [57]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보를 추출하여 Series로 반환
    # 반환된 user_rating은 title을 index로 가지는 Series객체이다.
    user_rating=ratings_matrix.loc[userId,:]
    
    # 이미 본 영화를 추출하여 list로 
    already_seen = user_rating[user_rating>0].index.tolist()
    # 모든 영화명을 list로
    movies_list = ratings_matrix.columns.tolist()
    # movies_list에서 already_seen 제외
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [58]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [59]:
unseen_list = get_unseen_movies(ratings_matrix, 9)

recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, 
                             columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


# PART 3 파이썬 추천 시스템 패키지 - Surprise
- 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 
- 다양한 추천 알고리즘 구축 가능 
- 사이킷런과 유사한 API와 프레임워크 제공 
- fit(), predict(), train_test_split(), cross_validate(), GridSearchCV


In [60]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

 Dataset 클래스를 이용한 데이터 로딩 
    - userID, movieID, rating과 같은 주요 데이터가 로우 레벨 형태로 돼있는 데이터 형식만 처리 
    
#### 1.  Dataset 클래스의 load_builtin()을 통해 무비렌즈 사이트에서 제공하는 데이터 내려받기

In [95]:
data = Dataset.load_builtin('ml-100k')
# 수행시마다 동일하게 데이터를 분할하기 위해 random_state값 부여
trainset,testset = train_test_split(data, test_size = 0.25, random_state=0)

In [96]:
# 모델학습-SVD로 잠재요인 협업 필터링
algo = SVD()
algo.fit(trainset)

test( )메서드 
- 사용자-아이템 평점 데이터 세트 전체에 대한 추천 예측

uid : user id /  
iid : item id /  
r_ui : rating /  
est : 예측값 

In [97]:
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.593177276101214, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.746533928749696, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.316750570977739, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.248009264407962, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.7194394325492977, details={'was_impossible': False})]

In [98]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.593177276101214),
 ('882', '291', 3.746533928749696),
 ('535', '507', 4.316750570977739)]

predict( )메서드 
- 개별 사용자의 아이템에 대한 추천 평점 예측하여 예측하여 est로 반환

- 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict( )를 반복적으로 수행한 결과 = test()

In [99]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.

uid = str(196)
lid = str(302)
pred = algo.predict(uid, lid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.11   {'was_impossible': False}


In [100]:
accuracy.rmse(predictions)

RMSE: 0.9514


0.9513550722009992

###  surprise 주요 모듈 소개

#### 2. OS 파일 데이터를 Surprise 데이터 세트로 로딩 
Reader클래스를 이용한 파싱 포맷을 정의해야 한다.

- line_format(string) 칼럼을 순서대로 나열. 공백으로 분리하여 칼럼으로 인식 
- sep(char) 칼럼 분리자, 판다스 DataFrame에서 입력받는 경우는 불필요 
- rating_scale(tuple, optional) 평점 값의 최소~최대 평점 설정. 디폴트는 (1,5)


In [102]:
import pandas as pd

ratings=pd.read_csv('ml-latest-small/ratings.csv')
# 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('ml-latest-small/ratings_noh.csv', index=False, header=False)

In [103]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ml-latest-small/ratings_noh.csv', reader = reader)

In [104]:
#  SVD 행렬 분해 기법을 이용한 추천 예측

trainset, testset = train_test_split(data, test_size = 0.25, random_state=0)

# 수행시마다 동일한 결과를 도출하기 위해 randcom_state 설정
# n_factor = SVD 행렬분해기법을 이용한 예측 잠재요인 K
algo = SVD(n_factors=50, random_state=0) 

# 학습 데이터 셋으로 학습하고 나서 테스트 데이터셋으로 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### 3. 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
- load_from_df()를 할 때에, 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야한다.

In [105]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame 에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야한다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader)
trainset, testset = train_test_split(data, test_size = 25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

# 위와 똑같은 결과가 나옴.

RMSE: 0.7940


0.7939608260279408

교차 검증과 하이퍼 파라미터 튜닝

In [81]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8670  0.8783  0.8701  0.8751  0.8759  0.8733  0.0041  
MAE (testset)     0.6665  0.6746  0.6692  0.6721  0.6728  0.6711  0.0029  
Fit time          7.88    8.30    8.20    8.44    7.84    8.13    0.23    
Test time         0.28    0.19    0.20    0.21    0.19    0.21    0.03    


{'test_rmse': array([0.8670291 , 0.87833582, 0.87008398, 0.87512376, 0.87593162]),
 'test_mae': array([0.66653309, 0.6746231 , 0.66917876, 0.67211174, 0.67282959]),
 'fit_time': (7.875079870223999,
  8.297083139419556,
  8.204076528549194,
  8.437147855758667,
  7.844844579696655),
 'test_time': (0.2812159061431885,
  0.18863296508789062,
  0.20311546325683594,
  0.2124631404876709,
  0.18952465057373047)}

In [82]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8744584943422478
{'n_epochs': 20, 'n_factors': 50}


### 6. Surprise를 이용한 개인화 영화추천 시스템 구축

DatasetAutoFolds 클래스 이용하여 데이터 세트 전체를 학습데이터로 사용


In [83]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [84]:

algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

userID=9인 사용자가 평점을 매기지 않은 영화 movieID=42에 대해 예측


In [85]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# 아이디 9가 영화42를 봤는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [86]:
# predict() 메서드 이용 userID와 movieID 문자열로 입력

uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


== > 위의 과정을 함수로 생성!
#### 평점 매기지 않은 전체 영화에 대한 예측 평점 순으로 영화 추천 
1. get_unseen_surprise() : 아이디별 평점 없는 영화 정보 반환 

2. recomm_movie_by_surprise() : 추천 대상 영화들에 대해 predict() 메서드 호출 
        → prediction 객체를 리스트로 저장 
        → top n개 추출하여 반환

In [87]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)
# 전체영화수 - 평점 매긴 영화 수 = 추천 대상 영화수

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [107]:
# 보지않은 영화에 대해 top10개만 추천. 

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬하고, top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [88]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
